# Fraud Detection with Amazon SageMaker FeatureStore

Kernel `conda_python3` or `Python 3 (Data Science) - if using SageMaker Studio` works well with this notebook.

The following policies need to be attached to the execution role:
- AmazonSageMakerFullAccess
- AmazonS3FullAccess

## Contents
1. [Background](#Background)
1. [Setup SageMaker FeatureStore](#Setup-SageMaker-FeatureStore)
1. [Inspect Dataset](#Inspect-Dataset)
1. [Ingest Data into FeatureStore](#Ingest-Data-into-FeatureStore)
1. [Build Training Dataset](#Build-Training-Dataset)
1. [Train and Deploy the Model](#Train-and-Deploy-the-Model)
1. [SageMaker FeatureStore At Inference](#SageMaker-FeatureStore-During-Inference)
1. [Cleanup Resources](#Cleanup-Resources)

## Background

Amazon SageMaker FeatureStore is a SageMaker capability that makes it easy for customers to create and manage curated data for machine learning (ML) development. SageMaker FeatureStore enables data ingestion via a high TPS API and data consumption via the online and offline stores. 

This notebook provides an example for the APIs provided by SageMaker FeatureStore by walking through the process of training a fraud detection model. The notebook demonstrates how the dataset's tables can be ingested into the FeatureStore, queried to create a training dataset, and quickly accessed during inference. 




![FeatureStore-1.png](notebook_images/FeatureStore-1.png)

![FeatureStore-2.png](notebook_images/FeatureStore-2.png)

![FeatureStore-3.png](notebook_images/FeatureStore-3.png)

![FeatureStore-4.png](notebook_images/FeatureStore-4.png)

![FeatureStore-5.png](notebook_images/FeatureStore-5.png)

### Terminology

A **FeatureGroup** is the main resource that contains the metadata for all the data stored in SageMaker FeatureStore. A FeatureGroup contains a list of FeatureDefinitions. A **FeatureDefinition** consists of a name and one of the following data types: a integral, string or decimal. The FeatureGroup also contains an **OnlineStoreConfig** and an **OfflineStoreConfig** controlling where the data is stored. Enabling the online store allows quick access to the latest value for a Record via the GetRecord API. The offline store, a required configuration, allows storage of historical data in your S3 bucket. 

Once a FeatureGroup is created, data can be added as Records. **Records** can be thought of as a row in a table. Each record will have a unique **RecordIdentifier** along with values for all other FeatureDefinitions in the FeatureGroup. 

In [43]:
#run time measurement. Can be removed
from datetime import datetime, timedelta, timezone
import time
tic=time.time()

## Setup SageMaker FeatureStore

Let's start by setting up the SageMaker Python SDK and boto client. Note that this notebook requires a `boto3` version above `1.17.21`

In [44]:
import boto3
import sagemaker

original_boto3_version = boto3.__version__
%pip install 'boto3>1.17.21'

Note: you may need to restart the kernel to use updated packages.


In [45]:
from sagemaker.session import Session

region = boto3.Session().region_name

boto_session = boto3.Session(region_name=region)

sagemaker_client = boto_session.client(service_name="sagemaker", region_name=region)
featurestore_runtime = boto_session.client(
    service_name="sagemaker-featurestore-runtime", region_name=region
)

feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime,
)

#### S3 Bucket Setup For The OfflineStore

SageMaker FeatureStore writes the data in the OfflineStore of a FeatureGroup to a S3 bucket owned by you. To be able to write to your S3 bucket, SageMaker FeatureStore assumes an IAM role which has access to it. The role is also owned by you.
Note that the same bucket can be re-used across FeatureGroups. Data in the bucket is partitioned by FeatureGroup.

Set the default s3 bucket name and it will be referenced throughout the notebook.

In [46]:
# You can modify the following to use a bucket of your choosing
default_s3_bucket_name = feature_store_session.default_bucket()
prefix = "sagemaker-featurestore-demo"

print(default_s3_bucket_name)

sagemaker-us-west-2-328296961357


Set up the IAM role. This role gives SageMaker FeatureStore access to your S3 bucket. 

<div class="alert alert-block alert-warning">
<b>Note:</b> In this example we use the default SageMaker role, assuming it has both <b>AmazonSageMakerFullAccess</b> and <b>AmazonSageMakerFeatureStoreAccess</b> managed policies. If not, please make sure to attach them to the role before proceeding.
</div>

In [47]:
from sagemaker import get_execution_role

# You can modify the following to use a role of your choosing. See the documentation for how to create this.
role = get_execution_role()
print(role)

arn:aws:iam::328296961357:role/service-role/AmazonSageMaker-ExecutionRole-20191125T182032


## Inspect Dataset

The provided dataset is a synthetic dataset with two tables: identity and transactions. They can both be joined by the `TransactionId` column. The transaction table contains information about a particular transaction such as amount, credit or debit card while the identity table contains information about the user such as device type and browser. The transaction must exist in the transaction table, but might not always be available in the identity table.

The objective of the model is to predict if a transaction is fraudulent or not, given the transaction record.

In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import io

s3_client = boto3.client("s3", region_name=region)

identity_data = pd.read_csv("data/sampled_identity.csv")
transaction_data = pd.read_csv("data/sampled_transactions.csv")

identity_data = identity_data.round(5)
transaction_data = transaction_data.round(5)

identity_data = identity_data.fillna(0)
transaction_data = transaction_data.fillna(0)

# Feature transformations for this dataset are applied before ingestion into FeatureStore.
# One hot encode card4, card6
encoded_card_bank = pd.get_dummies(transaction_data["card4"], prefix="card_bank")
encoded_card_type = pd.get_dummies(transaction_data["card6"], prefix="card_type")

transformed_transaction_data = pd.concat(
    [transaction_data, encoded_card_type, encoded_card_bank], axis=1
)
# blank space is not allowed in feature name
transformed_transaction_data = transformed_transaction_data.rename(
    columns={"card_bank_american express": "card_bank_american_express"}
)

In [49]:
identity_data.head()

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,...,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20
0,2990130,-5,38780.0,0.0,0.0,0.0,-70,0,1,100.0,...,32,80,253,241,260,125,T,F,F,T
1,2990266,-10,69246.0,0.0,0.0,0.0,-67,0,2,100.0,...,47,47,122,33,38,60,T,F,T,F
2,2992553,-45,348819.0,0.0,0.0,0.0,-73,0,0,100.0,...,21,143,268,111,2,135,F,F,T,F
3,2994568,-15,337170.0,0.0,0.0,0.0,-10,1,2,100.0,...,55,127,253,202,135,49,F,F,T,T
4,2994749,-5,680670.0,0.0,0.0,8.0,-1,2,2,100.0,...,52,43,257,7,19,254,F,F,T,T


In [50]:
transformed_transaction_data.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,card1,card2,card3,card4,card5,card6,...,N8,N9,card_type_0,card_type_credit,card_type_debit,card_bank_0,card_bank_american_express,card_bank_discover,card_bank_mastercard,card_bank_visa
0,3343087,0,8810855,29.00,12469,360.0,150.0,mastercard,126.0,debit,...,F,T,0,0,1,0,0,0,1,0
1,3307318,0,7955295,107.95,16188,178.0,150.0,mastercard,224.0,debit,...,F,T,0,0,1,0,0,0,1,0
2,3555327,0,15084339,159.95,1825,555.0,150.0,visa,226.0,debit,...,T,F,0,0,1,0,0,0,0,1
3,3310736,0,8017157,159.95,10057,225.0,150.0,mastercard,224.0,debit,...,F,F,0,0,1,0,0,0,1,0
4,3034711,0,1127470,117.00,11444,555.0,150.0,visa,226.0,debit,...,F,F,0,0,1,0,0,0,0,1


## Ingest Data into FeatureStore


In this step we will create the FeatureGroups representing the transaction and identity tables.

#### Define FeatureGroups

In [51]:
from time import gmtime, strftime, sleep

identity_feature_group_name = "identity-feature-group-" + strftime("%d-%H-%M-%S", gmtime())
transaction_feature_group_name = "transaction-feature-group-" + strftime("%d-%H-%M-%S", gmtime())

In [52]:
from sagemaker.feature_store.feature_group import FeatureGroup

identity_feature_group = FeatureGroup(
    name=identity_feature_group_name, sagemaker_session=feature_store_session
)
transaction_feature_group = FeatureGroup(
    name=transaction_feature_group_name, sagemaker_session=feature_store_session
)

In [53]:
import time

current_time_sec = int(round(time.time()))


def cast_object_to_string(data_frame):
    for label in data_frame.columns:
        if data_frame.dtypes[label] == "object":
            data_frame[label] = data_frame[label].astype("str").astype("string")


# cast object dtype to string. The SageMaker FeatureStore Python SDK will then map the string dtype to String feature type.
cast_object_to_string(identity_data)
cast_object_to_string(transformed_transaction_data)

# record identifier and event time feature names
record_identifier_feature_name = "TransactionID"
event_time_feature_name = "EventTime"

# append EventTime feature
identity_data[event_time_feature_name] = pd.Series(
    [current_time_sec] * len(identity_data), dtype="float64"
)
transformed_transaction_data[event_time_feature_name] = pd.Series(
    [current_time_sec] * len(transaction_data), dtype="float64"
)

# load feature definitions to the feature group. SageMaker FeatureStore Python SDK will auto-detect the data schema based on input data.
identity_feature_group.load_feature_definitions(data_frame=identity_data)
# output is suppressed
transaction_feature_group.load_feature_definitions(data_frame=transformed_transaction_data)
# output is suppressed

[FeatureDefinition(feature_name='TransactionID', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>),
 FeatureDefinition(feature_name='isFraud', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>),
 FeatureDefinition(feature_name='TransactionDT', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>),
 FeatureDefinition(feature_name='TransactionAmt', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='card1', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>),
 FeatureDefinition(feature_name='card2', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='card3', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='card4', feature_type=<FeatureTypeEnum.STRING: 'String'>),
 FeatureDefinition(feature_name='card5', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='card6', feature_type=<FeatureTypeEnum.STRING: 'String'>),
 FeatureDefinit

#### Create FeatureGroups in SageMaker FeatureStore

In [54]:
def wait_for_feature_group_creation_complete(feature_group):
    status = feature_group.describe().get("FeatureGroupStatus")
    while status == "Creating":
        print("Waiting for Feature Group Creation")
        time.sleep(5)
        status = feature_group.describe().get("FeatureGroupStatus")
    if status != "Created":
        raise RuntimeError(f"Failed to create feature group {feature_group.name}")
    print(f"FeatureGroup {feature_group.name} successfully created.")


identity_feature_group.create(
    s3_uri=f"s3://{default_s3_bucket_name}/{prefix}",
    record_identifier_name=record_identifier_feature_name,
    event_time_feature_name=event_time_feature_name,
    role_arn=role,
    enable_online_store=True,
)

transaction_feature_group.create(
    s3_uri=f"s3://{default_s3_bucket_name}/{prefix}",
    record_identifier_name=record_identifier_feature_name,
    event_time_feature_name=event_time_feature_name,
    role_arn=role,
    enable_online_store=True,
)

wait_for_feature_group_creation_complete(feature_group=identity_feature_group)
wait_for_feature_group_creation_complete(feature_group=transaction_feature_group)

Waiting for Feature Group Creation
Waiting for Feature Group Creation
Waiting for Feature Group Creation
Waiting for Feature Group Creation
FeatureGroup identity-feature-group-23-02-22-22 successfully created.
FeatureGroup transaction-feature-group-23-02-22-22 successfully created.


In [55]:
#run time measurement. Can be removed
toc=time.time()
elapsed_time = toc-tic
print(elapsed_time)
str(timedelta(seconds=elapsed_time))

26.059213876724243


'0:00:26.059214'

Confirm the FeatureGroup has been created by using the DescribeFeatureGroup and ListFeatureGroups APIs.

In [56]:
identity_feature_group.describe()

{'FeatureGroupArn': 'arn:aws:sagemaker:us-west-2:328296961357:feature-group/identity-feature-group-23-02-22-22',
 'FeatureGroupName': 'identity-feature-group-23-02-22-22',
 'RecordIdentifierFeatureName': 'TransactionID',
 'EventTimeFeatureName': 'EventTime',
 'FeatureDefinitions': [{'FeatureName': 'TransactionID',
   'FeatureType': 'Integral'},
  {'FeatureName': 'id_01', 'FeatureType': 'Integral'},
  {'FeatureName': 'id_02', 'FeatureType': 'Fractional'},
  {'FeatureName': 'id_03', 'FeatureType': 'Fractional'},
  {'FeatureName': 'id_04', 'FeatureType': 'Fractional'},
  {'FeatureName': 'id_05', 'FeatureType': 'Fractional'},
  {'FeatureName': 'id_06', 'FeatureType': 'Integral'},
  {'FeatureName': 'id_07', 'FeatureType': 'Integral'},
  {'FeatureName': 'id_08', 'FeatureType': 'Integral'},
  {'FeatureName': 'id_09', 'FeatureType': 'Fractional'},
  {'FeatureName': 'id_10', 'FeatureType': 'Integral'},
  {'FeatureName': 'id_11', 'FeatureType': 'Integral'},
  {'FeatureName': 'id_12', 'FeatureTyp

In [57]:
transaction_feature_group.describe()

{'FeatureGroupArn': 'arn:aws:sagemaker:us-west-2:328296961357:feature-group/transaction-feature-group-23-02-22-22',
 'FeatureGroupName': 'transaction-feature-group-23-02-22-22',
 'RecordIdentifierFeatureName': 'TransactionID',
 'EventTimeFeatureName': 'EventTime',
 'FeatureDefinitions': [{'FeatureName': 'TransactionID',
   'FeatureType': 'Integral'},
  {'FeatureName': 'isFraud', 'FeatureType': 'Integral'},
  {'FeatureName': 'TransactionDT', 'FeatureType': 'Integral'},
  {'FeatureName': 'TransactionAmt', 'FeatureType': 'Fractional'},
  {'FeatureName': 'card1', 'FeatureType': 'Integral'},
  {'FeatureName': 'card2', 'FeatureType': 'Fractional'},
  {'FeatureName': 'card3', 'FeatureType': 'Fractional'},
  {'FeatureName': 'card4', 'FeatureType': 'String'},
  {'FeatureName': 'card5', 'FeatureType': 'Fractional'},
  {'FeatureName': 'card6', 'FeatureType': 'String'},
  {'FeatureName': 'B1', 'FeatureType': 'Integral'},
  {'FeatureName': 'B2', 'FeatureType': 'Integral'},
  {'FeatureName': 'B3', '

In [58]:
sagemaker_client.list_feature_groups()  # use boto client to list FeatureGroups

{'FeatureGroupSummaries': [{'FeatureGroupName': 'transaction-feature-group-23-02-22-22',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-west-2:328296961357:feature-group/transaction-feature-group-23-02-22-22',
   'CreationTime': datetime.datetime(2022, 2, 23, 2, 22, 23, 930000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created'},
  {'FeatureGroupName': 'transaction-feature-group-22-23-10-10',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-west-2:328296961357:feature-group/transaction-feature-group-22-23-10-10',
   'CreationTime': datetime.datetime(2022, 2, 22, 23, 10, 12, 287000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created',
   'OfflineStoreStatus': {'Status': 'Active'}},
  {'FeatureGroupName': 'transaction-feature-group-22-22-36-55',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-west-2:328296961357:feature-group/transaction-feature-group-22-22-36-55',
   'CreationTime': datetime.datetime(2022, 2, 22, 22, 36, 56, 931000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created',
   'Of

#### PutRecords into FeatureGroup

After the FeatureGroups have been created, we can put data into the FeatureGroups by using the PutRecord API. This API can handle high TPS and is designed to be called by different streams. The data from all of these Put requests is buffered and written to S3 in chunks. The files will be written to the offline store within a few minutes of ingestion. For this example, to accelerate the ingestion process, we are specifying multiple workers to do the job simultaneously. It will take ~1min to ingest data to the 2 FeatureGroups, respectively.

In [59]:
identity_feature_group.ingest(data_frame=identity_data, max_workers=3, wait=True)

IngestionManagerPandas(feature_group_name='identity-feature-group-23-02-22-22', sagemaker_fs_runtime_client_config=<botocore.config.Config object at 0x7feb1c229c18>, max_workers=3, max_processes=1, profile_name=None, _async_result=<multiprocess.pool.MapResult object at 0x7feb0fa41d30>, _processing_pool=<pool ProcessPool(ncpus=1)>, _failed_indices=[])

In [60]:
transaction_feature_group.ingest(data_frame=transformed_transaction_data, max_workers=5, wait=True)

IngestionManagerPandas(feature_group_name='transaction-feature-group-23-02-22-22', sagemaker_fs_runtime_client_config=<botocore.config.Config object at 0x7feb1c229c18>, max_workers=5, max_processes=1, profile_name=None, _async_result=<multiprocess.pool.MapResult object at 0x7feb0faac358>, _processing_pool=<pool ProcessPool(ncpus=1)>, _failed_indices=[])

To confirm that data has been ingested, we can quickly retrieve a record from the online store:

In [61]:
record_identifier_value = str(2990130)

featurestore_runtime.get_record(
    FeatureGroupName=transaction_feature_group_name,
    RecordIdentifierValueAsString=record_identifier_value,
)

{'ResponseMetadata': {'RequestId': '1e08cf09-dc7e-485b-adad-cd2ba789d420',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '1e08cf09-dc7e-485b-adad-cd2ba789d420',
   'content-type': 'application/json',
   'content-length': '2636',
   'date': 'Wed, 23 Feb 2022 02:22:53 GMT'},
  'RetryAttempts': 0},
 'Record': [{'FeatureName': 'TransactionID', 'ValueAsString': '2990130'},
  {'FeatureName': 'isFraud', 'ValueAsString': '0'},
  {'FeatureName': 'TransactionDT', 'ValueAsString': '152647'},
  {'FeatureName': 'TransactionAmt', 'ValueAsString': '75.0'},
  {'FeatureName': 'card1', 'ValueAsString': '4577'},
  {'FeatureName': 'card2', 'ValueAsString': '583.0'},
  {'FeatureName': 'card3', 'ValueAsString': '150.0'},
  {'FeatureName': 'card4', 'ValueAsString': 'mastercard'},
  {'FeatureName': 'card5', 'ValueAsString': '219.0'},
  {'FeatureName': 'card6', 'ValueAsString': 'credit'},
  {'FeatureName': 'B1', 'ValueAsString': '69'},
  {'FeatureName': 'B2', 'ValueAsString': '80'},
  {'Featur

We can also retrieve a record of each feature group from the online store:

In [62]:
featurestore_runtime.batch_get_record(
    Identifiers=[
        {
            "FeatureGroupName": identity_feature_group_name,
            "RecordIdentifiersValueAsString": ["2990130"],
        },
        {
            "FeatureGroupName": transaction_feature_group_name,
            "RecordIdentifiersValueAsString": ["2990130"],
        },
    ]
)

{'ResponseMetadata': {'RequestId': '34192b03-b3c5-4380-bf69-96e015ef1f86',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '34192b03-b3c5-4380-bf69-96e015ef1f86',
   'content-type': 'application/json',
   'content-length': '3929',
   'date': 'Wed, 23 Feb 2022 02:22:53 GMT'},
  'RetryAttempts': 0},
 'Records': [{'FeatureGroupName': 'identity-feature-group-23-02-22-22',
   'RecordIdentifierValueAsString': '2990130',
   'Record': [{'FeatureName': 'TransactionID', 'ValueAsString': '2990130'},
    {'FeatureName': 'id_01', 'ValueAsString': '-5'},
    {'FeatureName': 'id_02', 'ValueAsString': '38780.0'},
    {'FeatureName': 'id_03', 'ValueAsString': '0.0'},
    {'FeatureName': 'id_04', 'ValueAsString': '0.0'},
    {'FeatureName': 'id_05', 'ValueAsString': '0.0'},
    {'FeatureName': 'id_06', 'ValueAsString': '-70'},
    {'FeatureName': 'id_07', 'ValueAsString': '0'},
    {'FeatureName': 'id_08', 'ValueAsString': '1'},
    {'FeatureName': 'id_09', 'ValueAsString': '100.0'},
    

The SageMaker Python SDK’s FeatureStore class also provides the functionality to generate Hive DDL commands. Schema of the table is generated based on the feature definitions. Columns are named after feature name and data-type are inferred based on feature type.

In [63]:
print(identity_feature_group.as_hive_ddl())

CREATE EXTERNAL TABLE IF NOT EXISTS sagemaker_featurestore.identity-feature-group-23-02-22-22 (
  TransactionID INT
  id_01 INT
  id_02 FLOAT
  id_03 FLOAT
  id_04 FLOAT
  id_05 FLOAT
  id_06 INT
  id_07 INT
  id_08 INT
  id_09 FLOAT
  id_10 INT
  id_11 INT
  id_12 INT
  id_13 INT
  id_14 INT
  id_15 INT
  id_16 INT
  id_17 STRING
  id_18 STRING
  id_19 STRING
  id_20 STRING
  EventTime FLOAT
  write_time TIMESTAMP
  event_time TIMESTAMP
  is_deleted BOOLEAN
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
  STORED AS
  INPUTFORMAT 'parquet.hive.DeprecatedParquetInputFormat'
  OUTPUTFORMAT 'parquet.hive.DeprecatedParquetOutputFormat'
LOCATION 's3://sagemaker-us-west-2-328296961357/sagemaker-featurestore-demo/328296961357/sagemaker/us-west-2/offline-store/identity-feature-group-23-02-22-22-1645582942/data'


In [64]:
print(transaction_feature_group.as_hive_ddl())

CREATE EXTERNAL TABLE IF NOT EXISTS sagemaker_featurestore.transaction-feature-group-23-02-22-22 (
  TransactionID INT
  isFraud INT
  TransactionDT INT
  TransactionAmt FLOAT
  card1 INT
  card2 FLOAT
  card3 FLOAT
  card4 STRING
  card5 FLOAT
  card6 STRING
  B1 INT
  B2 INT
  B3 INT
  B4 INT
  B5 INT
  B6 INT
  B7 INT
  B8 INT
  B9 INT
  B10 INT
  B11 INT
  B12 INT
  F1 INT
  F2 INT
  F3 INT
  F4 INT
  F5 INT
  F6 INT
  F7 INT
  F8 INT
  F9 INT
  F10 INT
  F11 INT
  F12 INT
  F13 INT
  F14 INT
  F15 INT
  F16 INT
  F17 INT
  N1 STRING
  N2 STRING
  N3 STRING
  N4 STRING
  N5 STRING
  N6 STRING
  N7 STRING
  N8 STRING
  N9 STRING
  card_type_0 INT
  card_type_credit INT
  card_type_debit INT
  card_bank_0 INT
  card_bank_american_express INT
  card_bank_discover INT
  card_bank_mastercard INT
  card_bank_visa INT
  EventTime FLOAT
  write_time TIMESTAMP
  event_time TIMESTAMP
  is_deleted BOOLEAN
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
  STORE

Now let's wait for the data to appear in our offline store before moving forward to creating a dataset. This will take approximately 5 minutes.

In [65]:
%%time
account_id = boto3.client("sts").get_caller_identity()["Account"]
print(account_id)

identity_feature_group_resolved_output_s3_uri = (
    identity_feature_group.describe()
    .get("OfflineStoreConfig")
    .get("S3StorageConfig")
    .get("ResolvedOutputS3Uri")
)
transaction_feature_group_resolved_output_s3_uri = (
    transaction_feature_group.describe()
    .get("OfflineStoreConfig")
    .get("S3StorageConfig")
    .get("ResolvedOutputS3Uri")
)

identity_feature_group_s3_prefix = identity_feature_group_resolved_output_s3_uri.replace(
    f"s3://{default_s3_bucket_name}/", ""
)
transaction_feature_group_s3_prefix = transaction_feature_group_resolved_output_s3_uri.replace(
    f"s3://{default_s3_bucket_name}/", ""
)

offline_store_contents = None
while offline_store_contents is None:
    objects_in_bucket = s3_client.list_objects(
        Bucket=default_s3_bucket_name, Prefix=transaction_feature_group_s3_prefix
    )
    if "Contents" in objects_in_bucket and len(objects_in_bucket["Contents"]) > 1:
        offline_store_contents = objects_in_bucket["Contents"]
    else:
        print("Waiting for data in offline store...\n")
        sleep(60)

print("Data available.")

328296961357
Waiting for data in offline store...

Waiting for data in offline store...

Waiting for data in offline store...

Waiting for data in offline store...

Waiting for data in offline store...

Waiting for data in offline store...

Data available.
CPU times: user 127 ms, sys: 5.7 ms, total: 133 ms
Wall time: 6min 1s


SageMaker FeatureStore adds metadata for each record that's ingested into the offline store.

In [66]:
#run time measurement. Can be removed
toc=time.time()
elapsed_time = toc-tic
print(elapsed_time)
str(timedelta(seconds=elapsed_time))

397.8773891925812


'0:06:37.877389'

## Build Training Dataset

SageMaker FeatureStore automatically builds the Glue Data Catalog for FeatureGroups (you can optionally turn it on/off while creating the FeatureGroup). In this example, we want to create one training dataset with FeatureValues from both identity and transaction FeatureGroups. This is done by utilizing the auto-built Catalog. We run an Athena query that joins the data stored in the offline store in S3 from the 2 FeatureGroups. 

In [67]:
identity_query = identity_feature_group.athena_query()
transaction_query = transaction_feature_group.athena_query()

identity_table = identity_query.table_name
transaction_table = transaction_query.table_name

query_string = (
    'SELECT * FROM "'
    + transaction_table
    + '" LEFT JOIN "'
    + identity_table
    + '" ON "'
    + transaction_table
    + '".transactionid = "'
    + identity_table
    + '".transactionid'
)
print("Running " + query_string)

# run Athena query. The output is loaded to a Pandas dataframe.
# dataset = pd.DataFrame()
identity_query.run(
    query_string=query_string,
    output_location="s3://" + default_s3_bucket_name + "/" + prefix + "/query_results/",
)
identity_query.wait()
dataset = identity_query.as_dataframe()

dataset

Running SELECT * FROM "transaction-feature-group-23-02-22-22-1645582943" LEFT JOIN "identity-feature-group-23-02-22-22-1645582942" ON "transaction-feature-group-23-02-22-22-1645582943".transactionid = "identity-feature-group-23-02-22-22-1645582942".transactionid


,transactionid,isfraud,transactiondt,transactionamt,card1,card2,card3,card4,card5,card6,...,id_15,id_16,id_17,id_18,id_19,id_20,eventtime.1,write_time.1,api_invocation_time.1,is_deleted.1
0,3246125,0,6212811,107.95,17185,327.0,150.0,discover,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3205708,0,5105182,141.00,15111,310.0,150.0,mastercard,224.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3371106,0,9639490,59.00,15986,360.0,150.0,mastercard,229.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3203818,0,5067035,56.50,8913,555.0,150.0,visa,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3283895,0,7333812,1017.95,7585,553.0,150.0,visa,226.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,3278974,0,7189573,59.00,15066,170.0,150.0,mastercard,102.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996,3236348,0,5944530,49.00,12695,490.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997,3440451,0,11578448,43.97,12581,453.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998,3384275,0,10007996,97.00,12501,490.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
# Prepare query results for training.
query_execution = identity_query.get_query_execution()
query_result = (
    "s3://"
    + default_s3_bucket_name
    + "/"
    + prefix
    + "/query_results/"
    + query_execution["QueryExecution"]["QueryExecutionId"]
    + ".csv"
)
print(query_result)

# Select useful columns for training with target column as the first.
dataset = dataset[
    [
        "isfraud",
        "transactiondt",
        "transactionamt",
        "card1",
        "card2",
        "card3",
        "card5",
        "card_type_credit",
        "card_type_debit",
        "card_bank_american_express",
        "card_bank_discover",
        "card_bank_mastercard",
        "card_bank_visa",
        "id_01",
        "id_02",
        "id_03",
        "id_04",
        "id_05",
    ]
]

# Write to csv in S3 without headers and index column.
dataset.to_csv("dataset.csv", header=False, index=False)
s3_client.upload_file("dataset.csv", default_s3_bucket_name, prefix + "/training_input/dataset.csv")
dataset_uri_prefix = "s3://" + default_s3_bucket_name + "/" + prefix + "/training_input/"

dataset

s3://sagemaker-us-west-2-328296961357/sagemaker-featurestore-demo/query_results/42fcfffb-c9f2-4dc3-9813-b88ac66ed31d.csv


,isfraud,transactiondt,transactionamt,card1,card2,card3,card5,card_type_credit,card_type_debit,card_bank_american_express,card_bank_discover,card_bank_mastercard,card_bank_visa,id_01,id_02,id_03,id_04,id_05
0,0,6212811,107.95,17185,327.0,150.0,166.0,0,1,0,1,0,0,NaN,NaN,NaN,NaN,NaN
1,0,5105182,141.00,15111,310.0,150.0,224.0,0,1,0,0,1,0,NaN,NaN,NaN,NaN,NaN
2,0,9639490,59.00,15986,360.0,150.0,229.0,0,1,0,0,1,0,NaN,NaN,NaN,NaN,NaN
3,0,5067035,56.50,8913,555.0,150.0,166.0,0,1,0,0,0,1,NaN,NaN,NaN,NaN,NaN
4,0,7333812,1017.95,7585,553.0,150.0,226.0,1,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0,7189573,59.00,15066,170.0,150.0,102.0,1,0,0,0,1,0,NaN,NaN,NaN,NaN,NaN
1996,0,5944530,49.00,12695,490.0,150.0,226.0,0,1,0,0,0,1,NaN,NaN,NaN,NaN,NaN
1997,0,11578448,43.97,12581,453.0,150.0,226.0,0,1,0,0,0,1,NaN,NaN,NaN,NaN,NaN
1998,0,10007996,97.00,12501,490.0,150.0,226.0,0,1,0,0,0,1,NaN,NaN,NaN,NaN,NaN


In [69]:
#run time measurement. Can be removed
toc=time.time()
elapsed_time = toc-tic
print(elapsed_time)
str(timedelta(seconds=elapsed_time))

403.7493300437927


'0:06:43.749330'

## Train and Deploy the Model

Now it's time to launch a Training job to fit our model. We use the gradient boosting algorithm provided by XGBoost libary to fit our data, call the SageMaker XGBoost container and construct a generic SageMaker estimator.
To save time, we will skip the training step and use the model that has been previously trained on this dataset already. You can view the training steps in sagemaker_featurestore_fraud_detection_python_sdk_long.ipynb notebook.

## Set up Hosting for the Model

In [70]:
import os
s3_model_path=feature_store_session.upload_data(path="model", bucket=default_s3_bucket_name, key_prefix=prefix + "/model")
model_artifacts=os.path.join(s3_model_path, "model.tar.gz")

In [71]:
from sagemaker import model

# retrieve xgboost docker container image from AWA ECR (Elastic Container Registry)
image_uri = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region=region,
    version="1.0-1",
    py_version="py3"
)

print(image_uri)
print(model_artifacts)

#Initialize SageMaker model.
training_model = model.Model(
    image_uri=image_uri, 
    model_data=model_artifacts,
    role=role,
    sagemaker_session=feature_store_session,
#    vpc_config = vpc_confg,
#    enable_network_isolation = False,
#    model_kms_key=model_kms_key,
)

246618743249.dkr.ecr.us-west-2.amazonaws.com/sagemaker-xgboost:1.0-1-cpu-py3
s3://sagemaker-us-west-2-328296961357/sagemaker-featurestore-demo/model/model.tar.gz


In [72]:
%%time
#Deploy SageMaker model on an endpoint.
from time import gmtime, strftime
realtime_endpoint_name = "xgboost-ep-sdk-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
predictor=training_model.deploy(
    initial_instance_count = 1,
    instance_type='ml.c4.xlarge',
    endpoint_name=realtime_endpoint_name,
#    serializer,
#    deserializer,
#    kms_key=None,
#    wait=True,
    )

-------!CPU times: user 98.8 ms, sys: 16.9 ms, total: 116 ms
Wall time: 3min 31s


In [73]:
#run time measurement. Can be removed
toc=time.time()
elapsed_time = toc-tic
print(elapsed_time)
str(timedelta(seconds=elapsed_time))

615.346474647522


'0:10:15.346475'

## SageMaker FeatureStore During Inference

SageMaker FeatureStore can be useful in supplementing data for inference requests because of the low-latency GetRecord functionality. For this demo, we will be given a TransactionId and query our online FeatureGroups for data on the transaction to build our inference request. 


In [74]:
# Incoming inference request.
transaction_id = str(3450774)

# Helper to parse the feature value from the record.
def get_feature_value(record, feature_name):
    return str(list(filter(lambda r: r["FeatureName"] == feature_name, record))[0]["ValueAsString"])


transaction_response = featurestore_runtime.get_record(
    FeatureGroupName=transaction_feature_group_name, RecordIdentifierValueAsString=transaction_id
)
transaction_record = transaction_response["Record"]

transaction_test_data = [
    get_feature_value(transaction_record, "TransactionDT"),
    get_feature_value(transaction_record, "TransactionAmt"),
    get_feature_value(transaction_record, "card1"),
    get_feature_value(transaction_record, "card2"),
    get_feature_value(transaction_record, "card3"),
    get_feature_value(transaction_record, "card5"),
    get_feature_value(transaction_record, "card_type_credit"),
    get_feature_value(transaction_record, "card_type_debit"),
    get_feature_value(transaction_record, "card_bank_american_express"),
    get_feature_value(transaction_record, "card_bank_discover"),
    get_feature_value(transaction_record, "card_bank_mastercard"),
    get_feature_value(transaction_record, "card_bank_visa"),
]

identity_response = featurestore_runtime.get_record(
    FeatureGroupName=identity_feature_group_name, RecordIdentifierValueAsString=transaction_id
)
identity_record = identity_response["Record"]
id_test_data = [
    get_feature_value(identity_record, "id_01"),
    get_feature_value(identity_record, "id_02"),
    get_feature_value(identity_record, "id_03"),
    get_feature_value(identity_record, "id_04"),
    get_feature_value(identity_record, "id_05"),
]

# Join all pieces for inference request.
inference_request = []
inference_request.extend(transaction_test_data[:])
inference_request.extend(id_test_data[:])

inference_request

['11923451',
 '50.0',
 '12501',
 '490.0',
 '150.0',
 '226.0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '1',
 '-40',
 '20130.0',
 '0.0',
 '0.0',
 '16.0']

In [75]:
import boto3
runtime = boto3.client(service_name="sagemaker-runtime")
realtime_response_sdk = runtime.invoke_endpoint(
    EndpointName=realtime_endpoint_name,
    Body=(",".join(inference_request)),
    ContentType="text/csv",
)

print("fraud probability of the transaction: ")
print(realtime_response_sdk["Body"].read())

fraud probability of the transaction: 
b'0.8026058077812195'


## Cleanup Resources

In [76]:
training_model.delete_model()
feature_store_session.delete_endpoint(endpoint_name=realtime_endpoint_name)

In [77]:
identity_feature_group.delete()
transaction_feature_group.delete()

In [78]:
# restore original boto3 version
%pip install 'boto3=={}'.format(original_boto3_version)

/bin/sh: -c: line 0: syntax error near unexpected token `('
/bin/sh: -c: line 0: `/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install 'boto3=={}'.format(original_boto3_version)'
Note: you may need to restart the kernel to use updated packages.


In [79]:
#run time measurement. Can be removed
toc=time.time()
elapsed_time = toc-tic
print(elapsed_time)
str(timedelta(seconds=elapsed_time))

618.2421405315399


'0:10:18.242141'